In [1]:
import sys
sys.path.append("../..")

import random
from time import sleep
from typing import List, Dict, Callable, Optional
from itertools import permutations
from datetime import datetime, timedelta

import numpy as np
import pandas as pd
import pandas_ta as ta
from IPython.display import display
from lightweight_charts import Chart

import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sesto.indicators import SMA, EMA, RSI, ROC, MACD, BB, ATR
from sesto.constants import CURRENCIES, MT5Timeframe
import sesto.metatrader.data as mtd
from sesto.plot import plot_tradingview, plot_plotly
from sesto.backtester import Backtester, Trade
from sesto.utils import get_pnl_at_price ,calculate_position_size, get_price_at_pnl, calculate_commission, calculate_break_even_price, calculate_price_with_spread, calculate_liquidation_price

MetaTrader 5 initialized successfully.


In [2]:
print(f"Number of currencies: {len(CURRENCIES)}")
# Generate all unique pairs of currency symbols
pairs = list(permutations(CURRENCIES, 2))
print(f"Number of pairs: {len(pairs)}")

Number of currencies: 8
Number of pairs: 56


In [3]:
# concatenate pairs into a single list
PAIRS = [pair[0] + pair[1] for pair in pairs]

In [4]:
DISPLAY_SYMBOL = 'EURUSD'

TIMEFRAMES = [MT5Timeframe.M5, MT5Timeframe.M15, MT5Timeframe.H1, MT5Timeframe.H4]
DATA_FROM_DATE = datetime(2024, 4, 5)
DATA_TO_DATE = datetime(2024, 9, 19)

MA_PERIODS = [7, 14, 21]

CSM_PERIOD = 21

In [5]:
MAIN_TIMEFRAME = MT5Timeframe.H1

INITIAL_CAPITAL = 10000

CSM_TREND_LONG_THRESHOLD = 65
CSM_TREND_SHORT_THRESHOLD = 35
CSM_REVERSAL_SHORT_THRESHOLD = 90
CSM_REVERSAL_LONG_THRESHOLD = 10
RSI_OVERSOLD = 30
RSI_OVERBOUGHT = 70

CAPITAL_PER_TRADE = 200  # $200 per trade

TP_PNL_MULTIPLIER = 0.5
SL_PNL_MULTIPLIER = -0.25

TRAILING_STOP_STEPS = [
    {'trigger_tp_multiplier': 0.50, 'new_sl_pnl_multiplier': 0.15},
    {'trigger_tp_multiplier': 0.25, 'new_sl_pnl_multiplier': 0.05},
    {'trigger_tp_multiplier': 0.10, 'new_sl_pnl_multiplier': 0.01},
]

SPREAD_MULTIPLIER = 0.0001
LEVERAGE = 250

In [6]:
mtd.fetch_data_all_timeframes(PAIRS, TIMEFRAMES, DATA_FROM_DATE, DATA_TO_DATE)

Failed to fetch data for symbol: USDEUR on timeframe: MT5Timeframe.M5
No data fetched for pair: USDEUR on timeframe: MT5Timeframe.M5
Failed to fetch data for symbol: USDGBP on timeframe: MT5Timeframe.M5
No data fetched for pair: USDGBP on timeframe: MT5Timeframe.M5
Fetched data for pair: USDJPY on timeframe: MT5Timeframe.M5
Fetched data for pair: USDCHF on timeframe: MT5Timeframe.M5
Fetched data for pair: USDCAD on timeframe: MT5Timeframe.M5
Failed to fetch data for symbol: USDAUD on timeframe: MT5Timeframe.M5
No data fetched for pair: USDAUD on timeframe: MT5Timeframe.M5
Failed to fetch data for symbol: USDNZD on timeframe: MT5Timeframe.M5
No data fetched for pair: USDNZD on timeframe: MT5Timeframe.M5
Fetched data for pair: EURUSD on timeframe: MT5Timeframe.M5
Fetched data for pair: EURGBP on timeframe: MT5Timeframe.M5
Fetched data for pair: EURJPY on timeframe: MT5Timeframe.M5
Fetched data for pair: EURCHF on timeframe: MT5Timeframe.M5
Fetched data for pair: EURCAD on timeframe: MT5T

In [7]:
# Remove symbols with no data for any timeframe
for timeframe in TIMEFRAMES:
    empty_symbols = [symbol for symbol, df in mtd.data[timeframe].items() if df is None or df.empty]
    for symbol in empty_symbols:
        del mtd.data[timeframe][symbol]
        if symbol in PAIRS:
            PAIRS.remove(symbol)

print(f"Remaining pairs after removing empty data: {len(PAIRS)}")

Remaining pairs after removing empty data: 28


In [8]:
def plot_data_availability():
    # Calculate expected number of rows for each timeframe
    expected_rows = {}
    for timeframe in TIMEFRAMES:
        if timeframe == MT5Timeframe.M5:
            minutes_per_candle = 5
        elif timeframe == MT5Timeframe.M15:
            minutes_per_candle = 15
        elif timeframe == MT5Timeframe.H1:
            minutes_per_candle = 60
        elif timeframe == MT5Timeframe.H4:
            minutes_per_candle = 240
        
        total_minutes = (DATA_TO_DATE - DATA_FROM_DATE).total_seconds() / 60
        expected_rows[timeframe] = int(total_minutes / minutes_per_candle)

    # Create a DataFrame to store the normalized row counts
    heatmap_data = []

    for timeframe in TIMEFRAMES:
        for symbol in PAIRS:
            if symbol in mtd.data[timeframe]:
                actual_rows = len(mtd.data[timeframe][symbol])
                normalized_value = actual_rows / expected_rows[timeframe]
                heatmap_data.append({
                    'Timeframe': timeframe.name,
                    'Symbol': symbol,
                    'Normalized Row Count': normalized_value
                })

    heatmap_df = pd.DataFrame(heatmap_data)

    # Aggregate data to handle potential duplicates
    heatmap_df = heatmap_df.groupby(['Symbol', 'Timeframe'])['Normalized Row Count'].mean().reset_index()

    # Pivot the DataFrame for the heatmap
    heatmap_pivot = heatmap_df.pivot(index='Symbol', columns='Timeframe', values='Normalized Row Count')

    # Create the heatmap using Plotly with a dark theme
    fig = go.Figure(data=go.Heatmap(
        z=heatmap_pivot.values,
        x=heatmap_pivot.columns,
        y=heatmap_pivot.index,
        colorscale='Twilight',
        colorbar=dict(
            title='Normalized Row Count',
            titleside='right',
            thickness=18,
            tickfont=dict(color='white'),
            titlefont=dict(color='white')
        ),
        text=heatmap_pivot.values,
        texttemplate='%{text:.2f}',
        textfont={"size": 12, "color": "black"},
    ))

    # Update layout with dark theme and improved spacing
    fig.update_layout(
        title={
            'text': 'Normalized Row Count Heatmap: Actual vs Expected',
            'y': 0.95,
            'x': 0.5,
            'xanchor': 'center',
            'yanchor': 'top'
        },
        yaxis_title='Symbol',
        height=max(600, len(PAIRS) * 25),  # Minimum height of 600px
        width=900,
        paper_bgcolor='rgb(30,30,30)',
        plot_bgcolor='rgb(30,30,30)',
        font=dict(color='white', family="Inter"),
        title_font=dict(color='white', size=16),
        xaxis=dict(
            tickfont=dict(color='white', size=12),
            title_font=dict(color='white', size=16),
            side='top',
            tickangle=-45
        ),
        yaxis=dict(
            tickfont=dict(color='white', size=12),
            title_font=dict(color='white', size=16),
            automargin=True,
        ),
        margin=dict(l=125, r=50, t=100, b=50)
    )

    # Show the plot
    fig.show()

# plot_data_availability()

In [9]:

for timeframe, pairs_data in mtd.data.items():
    for pair, df in pairs_data.items():
        if df is not None and not df.empty:
            for period in MA_PERIODS:
                SMA(df, period)
                EMA(df, period)
                ROC(df, period)
                RSI(df, period)
                BB(df, period, 2)
                ATR(df, period)
    
            df['psm'] = df[f'rsi-{CSM_PERIOD}'] + df[f'roc-{CSM_PERIOD}'] / 2

display(mtd.data[MAIN_TIMEFRAME][DISPLAY_SYMBOL].head())

,time,open,high,low,close,tick_volume,spread,real_volume,sma-7,ema-7,...,bbu-14-2,bbl-14-2,sma-21,ema-21,roc-21,rsi-21,bbm-21-2,bbu-21-2,bbl-21-2,psm
0,2024-04-04 21:00:00,1.08599,1.08599,1.08465,1.08499,2881,2,0,NaN,1.084990,...,NaN,NaN,NaN,1.084990,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-04-04 22:00:00,1.08498,1.08506,1.08332,1.08335,3437,1,0,NaN,1.084580,...,NaN,NaN,NaN,1.084841,NaN,NaN,NaN,NaN,NaN,NaN
2,2024-04-04 23:00:00,1.08335,1.08383,1.08316,1.08377,865,2,0,NaN,1.084377,...,NaN,NaN,NaN,1.084744,NaN,NaN,NaN,NaN,NaN,NaN
3,2024-04-05 00:00:00,1.08350,1.08399,1.08337,1.08375,1598,13,0,NaN,1.084221,...,NaN,NaN,NaN,1.084653,NaN,NaN,NaN,NaN,NaN,NaN
4,2024-04-05 01:00:00,1.08369,1.08398,1.08354,1.08362,659,4,0,NaN,1.084070,...,NaN,NaN,NaN,1.084559,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
def plot_indicator_availability():
    # Calculate expected number of rows for each timeframe
    expected_rows = {}
    for timeframe in TIMEFRAMES:
        if timeframe == MT5Timeframe.M5:
            minutes_per_candle = 5
        elif timeframe == MT5Timeframe.M15:
            minutes_per_candle = 15
        elif timeframe == MT5Timeframe.H1:
            minutes_per_candle = 60
        elif timeframe == MT5Timeframe.H4:
            minutes_per_candle = 240
        
        total_minutes = (DATA_TO_DATE - DATA_FROM_DATE).total_seconds() / 60
        expected_rows[timeframe] = int(total_minutes / minutes_per_candle)

    # Create a DataFrame to store the normalized row counts
    heatmap_data = []

    for timeframe in TIMEFRAMES:
        for symbol in PAIRS:
            if symbol in mtd.data[timeframe]:
                df = mtd.data[timeframe][symbol]
                if df is not None and not df.empty:
                    # Check for the presence of indicators
                    indicator_columns = [col for col in df.columns if col.startswith(('sma-', 'ema-', 'rsi-', 'roc-', 'macd', 'bb'))]
                    if indicator_columns:
                        actual_rows = df[indicator_columns].dropna().shape[0]
                        normalized_value = actual_rows / expected_rows[timeframe]
                        heatmap_data.append({
                            'Timeframe': timeframe.name,
                            'Symbol': symbol,
                            'Normalized Indicator Count': normalized_value
                        })

    heatmap_df = pd.DataFrame(heatmap_data)

    # Aggregate data to handle potential duplicates
    heatmap_df = heatmap_df.groupby(['Symbol', 'Timeframe'])['Normalized Indicator Count'].mean().reset_index()

    # Pivot the DataFrame for the heatmap
    heatmap_pivot = heatmap_df.pivot(index='Symbol', columns='Timeframe', values='Normalized Indicator Count')

    # Create the heatmap using Plotly with a dark theme
    fig = go.Figure(data=go.Heatmap(
        z=heatmap_pivot.values,
        x=heatmap_pivot.columns,
        y=heatmap_pivot.index,
        colorscale='Twilight',
        colorbar=dict(
            title='Normalized Indicator Count',
            titleside='right',
            thickness=18,
            tickfont=dict(color='white'),
            titlefont=dict(color='white')
        ),
        text=heatmap_pivot.values,
        texttemplate='%{text:.2f}',
        textfont={"size": 12, "color": "black"},
    ))

    # Update layout with dark theme and improved spacing
    fig.update_layout(
        title={
            'text': 'Normalized Indicator Count Heatmap: Actual vs Expected',
            'y': 0.95,
            'x': 0.5,
            'xanchor': 'center',
            'yanchor': 'top'
        },
        yaxis_title='Symbol',
        height=max(600, len(PAIRS) * 25),  # Minimum height of 600px
        width=900,
        paper_bgcolor='rgb(30,30,30)',
        plot_bgcolor='rgb(30,30,30)',
        font=dict(color='white', family="Inter"),
        title_font=dict(color='white', size=16),
        xaxis=dict(
            tickfont=dict(color='white', size=12),
            title_font=dict(color='white', size=16),
            side='top',
            tickangle=-45
        ),
        yaxis=dict(
            tickfont=dict(color='white', size=12),
            title_font=dict(color='white', size=16),
            automargin=True,
        ),
        margin=dict(l=125, r=50, t=100, b=50)
    )

    # Show the plot
    fig.show()

# plot_indicator_availability()

In [11]:

for timeframe in TIMEFRAMES:
    for currency in CURRENCIES:
        # Initialize DataFrames to hold PSM values for the current currency as base and quote
        base_psm_df = pd.DataFrame()
        quote_psm_df = pd.DataFrame()
        
        # Iterate over all pair data for the current timeframe
        for symbol, df in mtd.data[timeframe].items():
            if df is None or df.empty:
                continue
            
            base_currency = symbol[:3]
            quote_currency = symbol[3:]
            
            if currency == base_currency:
                # Use PSM as is for base currency
                base_psm_df[symbol] = df['psm']
                base_psm_df['time'] = df['time']
            elif currency == quote_currency:
                # Invert PSM for quote currency
                quote_psm_df[symbol] = -df['psm']
                quote_psm_df['time'] = df['time']
        
        # Calculate base CSM
        if not base_psm_df.empty:
            base_csm_df = base_psm_df.groupby('time').mean()
            base_csm_df['csm'] = base_csm_df.mean(axis=1)
            base_csm_df['csm'] = base_csm_df['csm'].rolling(CSM_PERIOD).mean()
            min_csm = base_csm_df['csm'].min()
            max_csm = base_csm_df['csm'].max()
            base_csm_df['csm'] = 100 * (base_csm_df['csm'] - min_csm) / (max_csm - min_csm)
            
            # Calculate MACD for base CSM
            MACD(base_csm_df, 'csm', 12, 26, 9)
        
        # Calculate quote CSM
        if not quote_psm_df.empty:
            quote_csm_df = quote_psm_df.groupby('time').mean()
            quote_csm_df['csm'] = quote_csm_df.mean(axis=1)
            quote_csm_df['csm'] = quote_csm_df['csm'].rolling(CSM_PERIOD).mean()
            min_csm = quote_csm_df['csm'].min()
            max_csm = quote_csm_df['csm'].max()
            quote_csm_df['csm'] = 100 * (quote_csm_df['csm'] - min_csm) / (max_csm - min_csm)
            
            # Calculate MACD for quote CSM
            MACD(quote_csm_df, 'csm', 12, 26, 9)
        
        # Add CSM and MACD values to all relevant symbols in mtd.data
        for symbol in mtd.data[timeframe]:
            base_currency = symbol[:3]
            quote_currency = symbol[3:]
            
            if currency == base_currency and not base_csm_df.empty:
                mtd.data[timeframe][symbol]['base_csm'] = mtd.data[timeframe][symbol]['time'].map(base_csm_df['csm'])
                mtd.data[timeframe][symbol]['base_csm_macd'] = mtd.data[timeframe][symbol]['time'].map(base_csm_df['macd'])
                mtd.data[timeframe][symbol]['base_csm_macd_signal'] = mtd.data[timeframe][symbol]['time'].map(base_csm_df['macd-signal'])
                mtd.data[timeframe][symbol]['base_csm_macd_histogram'] = mtd.data[timeframe][symbol]['time'].map(base_csm_df['macd-histogram'])
                mtd.data[timeframe][symbol]['base_csm_rsi'] = ta.rsi(mtd.data[timeframe][symbol]['base_csm'], length=CSM_PERIOD)
                mtd.data[timeframe][symbol]['base_csm_roc'] = ta.roc(mtd.data[timeframe][symbol]['base_csm'], length=CSM_PERIOD)

            elif currency == quote_currency and not quote_csm_df.empty:

                mtd.data[timeframe][symbol]['quote_csm'] = mtd.data[timeframe][symbol]['time'].map(quote_csm_df['csm'])
                mtd.data[timeframe][symbol]['quote_csm_macd'] = mtd.data[timeframe][symbol]['time'].map(quote_csm_df['macd'])
                mtd.data[timeframe][symbol]['quote_csm_macd_signal'] = mtd.data[timeframe][symbol]['time'].map(quote_csm_df['macd-signal'])
                mtd.data[timeframe][symbol]['quote_csm_macd_histogram'] = mtd.data[timeframe][symbol]['time'].map(quote_csm_df['macd-histogram'])
                mtd.data[timeframe][symbol]['quote_csm_rsi'] = ta.rsi(mtd.data[timeframe][symbol]['quote_csm'], length=CSM_PERIOD)
                mtd.data[timeframe][symbol]['quote_csm_roc'] = ta.roc(mtd.data[timeframe][symbol]['quote_csm'], length=CSM_PERIOD)



In [12]:
def plot_random_data():
    # Choose a random timeframe and symbol
    random_timeframe = random.choice(TIMEFRAMES)
    random_symbol = random.choice(PAIRS)

    # Get the data for the chosen timeframe and symbol
    df = mtd.data[random_timeframe][random_symbol]

    # Create subplots
    fig = make_subplots(rows=6, cols=1, shared_xaxes=True, vertical_spacing=0.05,
                        subplot_titles=('Close Price + SMA', 'CSM Values', 'Base CSM MACD', 'Quote CSM MACD', 'RSI', 'ROC'))

    # 1. Close Price + SMA
    fig.add_trace(go.Scatter(x=df['time'], y=df['close'], name='Close Price'), row=1, col=1)
    fig.add_trace(go.Scatter(x=df['time'], y=df['sma-14'], name='SMA-14'), row=1, col=1)

    # 2. CSM Values
    fig.add_trace(go.Scatter(x=df['time'], y=df['base_csm'], name='Base CSM'), row=2, col=1)
    fig.add_trace(go.Scatter(x=df['time'], y=df['quote_csm'], name='Quote CSM'), row=2, col=1)

    # 3. Base CSM MACD
    fig.add_trace(go.Scatter(x=df['time'], y=df['base_csm_macd'], name='Base MACD'), row=3, col=1)
    fig.add_trace(go.Scatter(x=df['time'], y=df['base_csm_macd_signal'], name='Base Signal'), row=3, col=1)
    fig.add_bar(x=df['time'], y=df['base_csm_macd_histogram'], name='Base Histogram', row=3, col=1)

    # 4. Quote CSM MACD
    fig.add_trace(go.Scatter(x=df['time'], y=df['quote_csm_macd'], name='Quote MACD'), row=4, col=1)
    fig.add_trace(go.Scatter(x=df['time'], y=df['quote_csm_macd_signal'], name='Quote Signal'), row=4, col=1)
    fig.add_bar(x=df['time'], y=df['quote_csm_macd_histogram'], name='Quote Histogram', row=4, col=1)

    # 5. RSI
    fig.add_trace(go.Scatter(x=df['time'], y=df['rsi-14'], name='RSI-14'), row=5, col=1)

    # 6. ROC
    fig.add_trace(go.Scatter(x=df['time'], y=df['roc-14'], name='ROC-14'), row=6, col=1)

    # Update layout
    fig.update_layout(
        height=1500, 
        width=1600, 
        title_text=f"Chart for {random_symbol} ({random_timeframe.name})", 
        template='plotly_dark'
    )

    # Update x-axis to include range sl_priceider
    fig.update_xaxes(
        rangesl_priceider_visible=True,
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(count=1, label="YTD", step="year", stepmode="todate"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(step="all")
            ])
        ),
        row=6, col=1  # Apply to the bottom subplot
    )

    # Hide rangesl_priceider for all subplots except the bottom one
    for i in range(1, 6):
        fig.update_xaxes(rangesl_priceider_visible=False, row=i, col=1)

    fig.update_yaxes(title_text="Price", row=1, col=1)
    fig.update_yaxes(title_text="CSM", row=2, col=1)
    fig.update_yaxes(title_text="Base MACD", row=3, col=1)
    fig.update_yaxes(title_text="Quote MACD", row=4, col=1)
    fig.update_yaxes(title_text="RSI", row=5, col=1)
    fig.update_yaxes(title_text="ROC", row=6, col=1)

    # Show the plot
    fig.show()

# plot_random_data()

In [13]:
class MyStrategy(Backtester):    
    def entry_condition(
        self, 
        symbol: str, 
        time: datetime, 
        row: pd.Series, 
        open_trades: List[Trade], 
        closed_trades: List[Trade], 
        timeframe: MT5Timeframe,
    ) -> Optional[Dict]:
        # Check if we already have an open trade for this symbol
        if any(trade.symbol == symbol for trade in open_trades):
            return None

        base_csm = row.get('base_csm')
        quote_csm = row.get('quote_csm')
        rsi = row.get('rsi-14')
    
        trend_up = (
            base_csm > CSM_TREND_LONG_THRESHOLD and
            base_csm < CSM_REVERSAL_SHORT_THRESHOLD and
            quote_csm < CSM_TREND_SHORT_THRESHOLD and
            quote_csm > CSM_REVERSAL_LONG_THRESHOLD
        )

        trend_down = (
            base_csm < CSM_TREND_SHORT_THRESHOLD and 
            base_csm > CSM_REVERSAL_LONG_THRESHOLD and
            quote_csm > CSM_TREND_LONG_THRESHOLD and
            quote_csm < CSM_REVERSAL_SHORT_THRESHOLD
        )

        entry = None
        entry_price = row['close']
        size = calculate_position_size(capital=CAPITAL_PER_TRADE, leverage=LEVERAGE)
        fee = calculate_commission(position_size_usd=size) 

        if trend_up:
            print(f"{time} - {symbol} - ENTRY CONDITION - TREND_UP - CLOSE: ${row['close']:.3f} - BASE_CSM: {base_csm:.3f} - QUOTE_CSM: {quote_csm:.3f} - RSI: {rsi:.3f}")
            sl = get_price_at_pnl(pnl_multiplier=SL_PNL_MULTIPLIER, order_fee=fee, position_size_usd=size, leverage=LEVERAGE, entry_price=entry_price, type='long')
            tp = get_price_at_pnl(pnl_multiplier=TP_PNL_MULTIPLIER, order_fee=fee, position_size_usd=size, leverage=LEVERAGE, entry_price=entry_price, type='long')

            entry = {
                'entry_price': entry_price,
                'type': 'long',
                'capital': CAPITAL_PER_TRADE,
                'tp_price': tp,
                'sl_price': sl
            }
        elif trend_down:
            print(f"{time} - {symbol} - ENTRY CONDITION - TREND_DOWN - CLOSE: ${row['close']:.3f} - BASE_CSM: {base_csm:.3f} - QUOTE_CSM: {quote_csm:.3f} - RSI: {rsi:.3f}")
            sl = get_price_at_pnl(pnl_multiplier=SL_PNL_MULTIPLIER, order_fee=fee, position_size_usd=size, leverage=LEVERAGE, entry_price=entry_price, type='short')
            tp = get_price_at_pnl(pnl_multiplier=TP_PNL_MULTIPLIER, order_fee=fee, position_size_usd=size, leverage=LEVERAGE, entry_price=entry_price, type='short')

            entry = {
                'entry_price': entry_price,
                'type': 'short',
                'capital': CAPITAL_PER_TRADE,
                'tp_price': tp,
                'sl_price': sl
            }

        # Ensure no open trades for the symbol
        if entry and not any(trade.symbol == symbol for trade in open_trades):
            return entry

        return None

    def exit_condition(
        self, 
        trade: Trade, 
        time: datetime, 
        row: pd.Series, 
        open_trades: List[Trade], 
        closed_trades: List[Trade], 
        timeframe: MT5Timeframe
    ) -> bool:
        return False

    def trailing_stop(
        self, 
        trade: Trade, 
        time: datetime, 
        row: pd.Series, 
        open_trades: List[Trade], 
        closed_trades: List[Trade], 
        timeframe: MT5Timeframe
    ):
        """
        Implement a trailing stop that adjusts the stop loss to make the trade risk-free
        once the unrealized PnL exceeds 5% of the capital allocation.
        """
        
        for trailing_step in TRAILING_STOP_STEPS:
            trigger_tp_multiplier, new_sl_pnl_multiplier = trailing_step['trigger_tp_multiplier'], trailing_step['new_sl_pnl_multiplier']
            pnl_threshold = TP_PNL_MULTIPLIER * trigger_tp_multiplier * CAPITAL_PER_TRADE

            if trade.unrealized_pnl >= pnl_threshold:
                old_sl_price = trade.sl_price

                if trade.type == 'long':
                    new_sl_price = get_price_at_pnl(pnl_multiplier=new_sl_pnl_multiplier, order_fee=trade.order_fee, position_size_usd=trade.position_size_usd, leverage=trade.leverage, entry_price=trade.entry_price, type='long')
                    if new_sl_price > old_sl_price:
                        trade.sl_price = new_sl_price
                else:
                    new_sl_price = get_price_at_pnl(pnl_multiplier=new_sl_pnl_multiplier, order_fee=trade.order_fee, position_size_usd=trade.position_size_usd, leverage=trade.leverage, entry_price=trade.entry_price, type='short')
                    if new_sl_price < old_sl_price:
                        trade.sl_price = new_sl_price
                
                if trade.sl_price != old_sl_price:
                    capital_gain_at_current_unrealized_pnl = (trade.unrealized_pnl / trade.capital) * 100
                    old_sl_price_diff = (trade.entry_price / old_sl_price - 1) * 100
                    new_sl_price_diff = (trade.entry_price / trade.sl_price - 1) * 100
                    pnl_at_old_sl = get_pnl_at_price(current_price=old_sl_price, entry_price=trade.entry_price, position_size_usd=trade.position_size_usd, leverage=trade.leverage, type=trade.type)
                    pnl_at_new_sl = get_pnl_at_price(current_price=trade.sl_price, entry_price=trade.entry_price, position_size_usd=trade.position_size_usd, leverage=trade.leverage, type=trade.type)
                    capital_loss_at_old_sl = (pnl_at_old_sl / trade.capital) * 100
                    capital_loss_at_new_sl= (pnl_at_new_sl / trade.capital) * 100

                    print(f"{row['time']} - {trade.symbol} - TRAILING STOP - UNREALIZED PNL: ${trade.unrealized_pnl:.3f} ({capital_gain_at_current_unrealized_pnl:.3f}% CAP) - OLD SL: ${old_sl_price:.3f} ({old_sl_price_diff:.3f}%)({capital_loss_at_old_sl:.3f}% CAP) - NEW SL: ${trade.sl_price:.3f} ({new_sl_price_diff:.3f}%)({capital_loss_at_new_sl:.3f}% CAP)")
                    break  # Exit the loop after adjusting the stop loss


In [14]:
backtest = MyStrategy(
    mtd.data, 
    initial_capital=INITIAL_CAPITAL, 
    main_timeframe=MAIN_TIMEFRAME,
    spread_multiplier=SPREAD_MULTIPLIER,
    leverage=LEVERAGE
)

backtest.run()

2024-04-25 10:00:00 - USDJPY - ENTRY CONDITION - TREND_UP - CLOSE: $155.626 - BASE_CSM: 80.345 - QUOTE_CSM: 33.462 - RSI: 73.379
2024-04-25 10:00:00 - USDJPY - OPENED TRADE - long - ENTRY: $155.642 - TP: $155.937 (-0.190%) - SL: $155.470 (0.110%) - LIQ: $155.019 - BE: $155.647 - AVAILABLE CAPITAL: $9798.400
2024-04-25 15:00:00 - USDJPY - CLOSED TRADE - long - ENTRY: $155.642 - CLOSE: $155.470 - PNL: $-61.58 - SL: $155.470 - TP: $155.937 - REASON: SL - AVAILABLE CAPITAL: $9936.817
2024-04-25 15:00:00 - USDJPY - ENTRY CONDITION - TREND_UP - CLOSE: $155.678 - BASE_CSM: 73.105 - QUOTE_CSM: 24.510 - RSI: 78.827
2024-04-25 15:00:00 - USDJPY - OPENED TRADE - long - ENTRY: $155.694 - TP: $155.989 (-0.190%) - SL: $155.522 (0.110%) - LIQ: $155.071 - BE: $155.699 - AVAILABLE CAPITAL: $9735.217
2024-04-25 16:00:00 - USDJPY - CLOSED TRADE - long - ENTRY: $155.694 - CLOSE: $155.522 - PNL: $-61.58 - SL: $155.522 - TP: $155.989 - REASON: SL - AVAILABLE CAPITAL: $9873.635
2024-04-25 16:00:00 - USDJPY -

In [15]:
trade_log = pd.DataFrame(backtest.trade_log)
trade_log[['type', 'entry_price', 'tp_price', 'sl_price', 'liq_p','pnl', 'potential_loss_usd']]
# trade_log.to_csv('./log.csv')
trade_query = trade_log[trade_log['symbol'] == DISPLAY_SYMBOL]
trade_query.head()

,symbol,entry_time,entry_price,type,position_size_usd,tp_price,sl_price,capital,potential_profit_usd,potential_loss_usd,...,pnl,max_drawdown,max_profit,closing_reason,unrealized_pnl,leverage,liq_p,be_p,order_fee,spread_multiplier
406,EURUSD,2024-04-09 04:00:00,1.086059,long,50000,1.088122,1.086276,200,94.996900,54.988101,...,3.405399,0,0,SL,0,250,1.081714,1.086093,1.6,0.0001
407,EURUSD,2024-04-09 14:00:00,1.086229,long,50000,1.088292,1.085034,200,94.996900,54.988101,...,88.387401,0,0,TP,0,250,1.081884,1.086263,1.6,0.0001
408,EURUSD,2024-04-09 15:00:00,1.088149,long,50000,1.090216,1.086952,200,94.996900,54.988101,...,-61.582602,0,0,SL,0,250,1.083796,1.088184,1.6,0.0001
409,EURUSD,2024-04-10 23:00:00,1.074043,short,50000,1.072002,1.073398,200,95.015902,54.999100,...,23.409401,0,0,SL,0,250,1.078339,1.074008,1.6,0.0001
410,EURUSD,2024-04-12 20:00:00,1.064584,short,50000,1.062560,1.064370,200,95.015902,54.999100,...,3.407401,0,0,SL,0,250,1.068842,1.064549,1.6,0.0001


In [16]:
performance_report = backtest.generate_report()
performance_report
# performance_report.to_csv('./performance_report.csv')

,Metric,Value
0,Initial Capital,$10000.00
1,Final Capital,$1822.43
2,Total Profit,$-8177.57
3,Return (%),-81.78%
4,Annualized Return (%),-97.89%
5,Volatility (Ann.),46.41
6,Sharpe Ratio,-2.15
7,Sortino Ratio,-3.12
8,Calmar Ratio,1.16
9,Max. Drawdown ($),$-8413.25


In [17]:
trades_per_symbol = trade_log['symbol'].value_counts()

# Create a bar chart
fig = go.Figure(data=[go.Bar(
    x=trades_per_symbol.index,
    y=trades_per_symbol.values,
    text=trades_per_symbol.values,
    textposition='auto',
)])

# Update the layout
fig.update_layout(
    title='Number of Trades per Symbol',
    xaxis_title='Symbol',
    yaxis_title='Number of Trades',
    height=600,
    width=1000,
    template='plotly_dark'
)

# Show the plot
fig.show()

In [18]:
o_df = mtd.data[MAIN_TIMEFRAME][DISPLAY_SYMBOL]
df_first_row = o_df.iloc[:1]  # Selecting the first row
df = o_df.iloc[1:]       # Selecting the remaining rows

chart = Chart()
chart.set(df_first_row)
chart.show()

def create_horizontal_line(start_time, close_time, price, name):
    return pd.DataFrame({
        'time': [start_time, close_time],
        name: [price, price]
    })

def create_entry_to_close_line(start_time, end_time, entry_price, close_price, name):
    return pd.DataFrame({
        'time': [start_time, end_time],
        name: [entry_price, close_price]
    })

for i, row in df.iterrows():
    chart.update(row)

    trade_entry_query = trade_log[
        (trade_log['entry_time'] == row['time']) & 
        (trade_log['symbol'] == DISPLAY_SYMBOL)
    ]
    
    if not trade_entry_query.empty:
        trade = trade_entry_query.iloc[0]  # Get the first row
        chart.marker(color="white", text=f'{trade.type} opened at ${trade.entry_price:.3f}', position='below' if trade.type == 'long' else 'above', shape='arrow_up' if trade.type == 'long' else 'arrow_down')
        
        # Create TP line
        tp_line = chart.create_line('TP', color='green', style= 'dashed', price_label=False, price_line=False)
        tp_data = create_horizontal_line(trade.entry_time, trade.close_time, trade.tp_price, 'TP')
        tp_line.set(tp_data)
        
        # Create SL line
        sl_line = chart.create_line('SL', color='red', style="dashed", price_label=False, price_line=False)
        sl_data = create_horizontal_line(trade.entry_time, trade.close_time, trade.sl_price, 'SL')
        sl_line.set(sl_data)

        trade_line = chart.create_line('TRADE', color='white', price_label=False, price_line=False)
        trade_data = create_entry_to_close_line(trade.entry_time, trade.close_time, trade.entry_price, trade.close_price, 'TRADE')
        trade_line.set(trade_data)
    
    trade_close_query = trade_log[
        (trade_log['close_time'] == row['time']) & 
        (trade_log['symbol'] == DISPLAY_SYMBOL)
    ]

    if not trade_close_query.empty:
        trade = trade_close_query.iloc[0]  # Get the first row
        trade_text = f'Closed at ${trade.close_price:.3f} - PNL: {trade.pnl / trade.capital * 100:.3f}% - ${trade.pnl:.3f}'
        if trade.pnl > 0:
            chart.marker(color="green", position="inside", shape='circle', text=trade_text)
        else:
            chart.marker(color="red", position="inside", shape="circle", text=trade_text)


    # Conditional sleep based on main_timeframe
    if MAIN_TIMEFRAME == MT5Timeframe.M1:
        sleep(0.0025)
    elif MAIN_TIMEFRAME == MT5Timeframe.M5:
        sleep(0.005)
    elif MAIN_TIMEFRAME == MT5Timeframe.M15:
        sleep(0.01)
    elif MAIN_TIMEFRAME == MT5Timeframe.M30:
        sleep(0.03)
    elif MAIN_TIMEFRAME == MT5Timeframe.H1:
        sleep(0.05)


KeyboardInterrupt: 